### Read WebEOC Board Data with API

reference for WebEOC API: https://ghinternational-my.sharepoint.com/my?id=%2Fpersonal%2Feendrulat%5Fghinternational%5Fcom%2FDocuments%2FMicrosoft%20Teams%20Chat%20Files%2FWebEOC%20REST%20API%20Technical%20Guide%20Sep2022%2Epdf&parent=%2Fpersonal%2Feendrulat%5Fghinternational%5Fcom%2FDocuments%2FMicrosoft%20Teams%20Chat%20Files


In [2]:
import json, requests, time

In [3]:
# Set Default Incident and Position
position = "CMD Incident Commander"
incident = "Trial"
base_url = 'https://trial1.demo.webeocasp.com/trial1/api/rest.svc'

# Get username and password from config file 
with open('config.json') as config_file:
    config = json.load(config_file)

webeoc_username = config['webeoc_username']
webeoc_password = config['webeoc_password']


In [4]:
# Create the session to be utilized by the REST API.
def create_session(base_url):
	url = f"{base_url}/sessions"

	payload = json.dumps({
		"username": webeoc_username,
		"password": webeoc_password,
		"position": position,
		"incident": incident
	})
	headers = {
		'Content-Type': 'application/json',
	}
	session = requests.Session()
	session.post(url, headers=headers, data=payload)

	return session

# End the session.
def close_session(session):
  url = f"{base_url}/sessions"
  session.delete(url)

In [5]:
#### Retrieve list of WebEOC Boards
def get_webeoc_boards(base_url, session):
    # print('getting boards', base_url)
    boards_url = f"{base_url}/boards"
    
    response = session.get(boards_url)

    if response.status_code == 200:
		# Record created successfully, return the response
        return response.json()
    else:
		# Record creation failed, raise an exception with the error message
        response.raise_for_status()


# Retrieve list of display views for a WebEOC board
def get_board_display_views(base_url, session, board_name, view_type):

    display_views_url = f"{base_url}/boards/{board_name}/views/{view_type}"

    response = session.get(display_views_url)
    #response = requests.get(display_views_url, headers=headers, cookies=cookies)

    if response.status_code == 200:
        # Request successful, return the list of display views
        return response.json()
    else:
        # Request failed, raise an exception with the error message
        response.raise_for_status()

# Retrieve list of view fields  for a WebEOC board
def get_view_fields(base_url, session, board_name, viewName):
    views_fields_url = f"{base_url}/boards/{board_name}/views/{viewName}/fields"

    response = session.get(views_fields_url)
    #response = requests.get(display_views_url, headers=headers, cookies=cookies)

    if response.status_code == 200:
        # Request successful, return the list of display views
        return response.json()
    else:
        # Request failed, raise an exception with the error message
        response.raise_for_status()

# Retrieve records from selected WebEOC board
def get_board_records(base_url, session, board_name, displayviewname):

    records_url = f"{base_url}/board/{board_name}/display/{displayviewname}"
    print(records_url)
    response = session.get(records_url)

    if response.status_code == 200:
        # Request successful, return the list of records
        with open(f"{board_name}-{displayviewname}-{time.strftime('%Y-%m-%d')}.txt", 'w') as file:
            json.dump(response.json(), file, indent=4)
        return response.json()
    else:
        # Request failed, raise an exception with the error message
        response.raise_for_status()



In [69]:
def create_board_record(base_url, session, board_name, inputviewname, record_data):
	create_record_url = f"{base_url}/board/{board_name}/input/{inputviewname}"
	print (create_record_url)
	payload = {'data': json.dumps(record_data)}

	response = session.post(create_record_url, json=payload)

	if response.status_code == 201:
		# Record created successfully, return the response
		return response.json()
	else:
		# Record creation failed, raise an exception with the error message
		response.raise_for_status()
          


In [77]:
#Main function
def main():
	view_type = ["inputs","displays"] # inputs or displays

	session = create_session(base_url)

	try:
		boardResponse = get_webeoc_boards(base_url, session)

		for board in boardResponse:
			print(board)
			print('')
			try:
				for viewType in view_type:
					viewsResponse = get_board_display_views(base_url, session, board, viewType)
					for view in viewsResponse:
						print('.....',viewType,': ', view)
	
			except:
				print('error getting views', view)

	except requests.exceptions.RequestException as e:
		print("Failed to create record:", e)

	close_session(session)



In [85]:
main()

After Action Review

..... Input
..... Input Comment
..... Remove AAR
..... Remove Comment
..... (Version)
..... Blank Panel
..... Dashboard
..... Details
..... List - After Actions
..... List - After Actions (Global)
..... List - Improvement Plan
..... List - Improvement Plan (Global)
..... PDF - After Action Reviews
..... PDF - After Action Reviews (Global)
..... PDF - Improvement Plan
..... PDF - Improvement Plan (Global)
Checklists

..... Activated Checklist Delete
..... Activated Checklist Input
..... Activated Checklist Item Delete
..... Activated Checklist Item Input
..... Checklist Template Delete
..... Checklist Template Input
..... Checklist Template Item Delete
..... Checklist Template Item Input
..... Utility Activate Checklist
..... Utility Activate Checklist Item
..... Utility List Import
..... Utility Set Checklist Status
..... Utility Set Item Completion Status
..... Utility Set Item Not Applicable Status
..... Utility Set Item Not Started Status
..... Utility Set Item 

In [6]:
session = create_session(base_url)
response = get_view_fields(base_url, session, 'Incident Creator', 'Incident List')
print(response)
close_session(session)

['dataid', 'incidentid', 'userid', 'positionid', 'entrydate', 'subscribername', 'prevdataid', 'globalid', 'review_number', 'incident_activationSummary', 'incident_agency', 'incident_comments', 'incident_location', 'incident_point_of_contact', 'incident_contact_number', 'incident_isActive', 'webeoc_incident_name', 'incident_creationDate', 'incident_eventType', 'incident_eventTypeOther', 'incident_activationLevel', 'incident_activationType', 'incident_createdBy_user', 'incident_createdBy_position', 'location']


In [89]:
session = create_session(base_url)
response = get_board_records(base_url, session, 'Situation Report - Working', 'BoardScript Lifelines Data')
print('success')
close_session(session)

https://trial1.demo.webeocasp.com/trial1/api/rest.svc/board/Situation Report - Working/display/BoardScript Lifelines Data
success
